In [29]:
from EduKTM import DKT
import torch
import torch.utils.data as Data
import numpy as np
import logging
import tqdm
from sklearn.metrics import roc_auc_score

In [30]:
NUM_QUESTIONS = 9
BATCH_SIZE = 64
HIDDEN_SIZE = 10
NUM_LAYERS = 1

def get_data_loader(data_path, batch_size, shuffle=False):
    data = torch.FloatTensor(np.load(data_path))
    data_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    return data_loader

In [31]:
train_loader = get_data_loader(
    './train_data.npy', BATCH_SIZE, True)
test_loader = get_data_loader(
    './test_data.npy', BATCH_SIZE, False)

logging.getLogger().setLevel(logging.INFO)

dkt = DKT(NUM_QUESTIONS, HIDDEN_SIZE, NUM_LAYERS)

In [32]:
dkt = DKT(NUM_QUESTIONS, HIDDEN_SIZE, NUM_LAYERS)
dkt.train(train_loader, epoch=100)
dkt.save("dkt.params")


Epoch 0: 100%|██████████| 985/985 [00:08<00:00, 117.54it/s]


[Epoch 0] LogisticLoss: 19.118973


Epoch 1: 100%|██████████| 985/985 [00:08<00:00, 113.46it/s]


[Epoch 1] LogisticLoss: 14.254794


Epoch 2: 100%|██████████| 985/985 [00:08<00:00, 110.10it/s]


...

[Epoch 97] LogisticLoss: 12.099882


Epoch 98: 100%|██████████| 985/985 [00:09<00:00, 107.34it/s]


[Epoch 98] LogisticLoss: 12.126188


Epoch 99: 100%|██████████| 985/985 [00:09<00:00, 106.78it/s]
INFO:root:save parameters to dkt.params


[Epoch 99] LogisticLoss: 12.347364


In [33]:
dkt.load("dkt.params")
auc = dkt.eval(test_loader)
print("auc: %.6f" % auc)

/opt/homebrew/Caskroom/miniconda/base/envs/sql_task_adaptation/lib/python3.12/site-packages/EduKTM/DKT/DKT.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.dkt_mode

auc: 0.932941


In [34]:
def process_raw_pred(raw_question_matrix, raw_pred, num_questions: int) -> tuple:
    questions = torch.nonzero(raw_question_matrix)[1:, 1] % num_questions
    length = questions.shape[0]
    pred = raw_pred[: length]
    pred = pred.gather(1, questions.view(-1, 1)).flatten()
    truth = torch.nonzero(raw_question_matrix)[1:, 1] // num_questions
    
    return pred, truth

def eval(model, test_data) -> float:
    model.eval()
    y_pred = torch.Tensor([])
    y_truth = torch.Tensor([])
    for batch in tqdm.tqdm(test_data, "evaluating"):
        integrated_pred = model(batch)
        batch_size = batch.shape[0]
        for student in range(batch_size):
            pred, truth = process_raw_pred(batch[student], integrated_pred[student], NUM_QUESTIONS)
            y_pred = torch.cat([y_pred, pred])
            y_truth = torch.cat([y_truth, truth])
    return [y_pred, y_truth]

In [35]:
model = dkt.dkt_model

y_pred, y_truth = eval(model, test_loader)

print(y_pred[:10])
print(y_truth.shape)

y_pred_binary = (y_pred >= 0.5).float()
print("Binary predictions:", y_pred_binary)

print("Accuracy:", torch.sum(y_pred_binary == y_truth)/len(y_truth))
print("y_pred_binary shape", y_pred_binary.shape)
roc_auc_score(y_truth.detach().numpy(), y_pred.detach().numpy())

evaluating: 100%|██████████| 422/422 [00:05<00:00, 83.92it/s] 


tensor([0.6031, 0.5911, 0.5403, 0.2588, 0.4531, 0.5879, 0.4737, 0.4119, 0.6010,
        0.6472], grad_fn=<SliceBackward0>)
torch.Size([1323000])
Binary predictions: tensor([1., 1., 1.,  ..., 1., 1., 1.])
Accuracy: tensor(0.9127)
y_pred_binary shape torch.Size([1323000])


np.float64(0.93294126459603)

In [ ]:
model = dkt.dkt_model

first_batch = next(iter(test_loader))
first_batch_element = first_batch[0]
first_batch_size_1 = first_batch_element.unsqueeze(0)
model.eval()

pred = model(first_batch_size_1)

print(pred.shape)

last_pred = pred[0, -1, :]

print(pred)

# new_dataset = first_batch_element.unsqueeze(0)

# new_test_loader = Data.DataLoader(new_dataset, batch_size=1, shuffle=False)
# print(new_test_loader.dataset.shape)
# pred_y, true_y = eval(model, new_test_loader)
# print("Predictions:", pred_y)
# print("Ground truth:", true_y)
# print("ROC AUC:", roc_auc_score(true_y.detach().numpy(), pred_y.detach().numpy()))


torch.Size([1, 50, 9])
tensor([[[4.7285e-01, 6.0306e-01, 7.7191e-01, 5.2794e-01, 8.8285e-01,
          3.8078e-01, 3.4245e-01, 5.6543e-01, 8.6812e-01],
         [3.8474e-01, 7.4880e-01, 5.9109e-01, 4.8322e-02, 2.2856e-01,
          3.5398e-01, 7.9818e-03, 7.0463e-02, 8.7563e-01],
         [4.3054e-01, 2.9381e-01, 4.0341e-01, 5.4033e-01, 1.0015e-01,
          5.5437e-03, 2.8360e-02, 2.4416e-01, 1.1117e-01],
         [6.4683e-01, 6.7789e-01, 4.9755e-01, 7.9187e-01, 2.5879e-01,
          1.9562e-02, 8.3900e-01, 8.2422e-01, 4.9452e-03],
         [4.8529e-01, 6.7383e-01, 5.8220e-01, 9.5048e-01, 3.1449e-02,
          4.5310e-01, 8.2668e-01, 6.4696e-01, 5.1043e-01],
         [3.5370e-01, 5.7371e-01, 5.6845e-01, 9.8773e-01, 1.8883e-01,
          1.4651e-01, 5.8788e-01, 4.2280e-01, 9.9103e-01],
         [2.0328e-01, 7.0403e-01, 5.0446e-01, 9.7427e-01, 2.1935e-01,
          1.4610e-01, 6.7992e-01, 4.7367e-01, 9.5128e-01],
         [6.4558e-01, 6.3517e-01, 5.6925e-01, 9.4321e-01, 4.6463e-02,
    